In [13]:
#import important packages

import pandas as pd
import numpy as np
import binance as bn
from binance.client import Client
from binance import ThreadedWebsocketManager
import json
import websocket
import psycopg2
from datetime import datetime 

In [14]:
import snowflake.connector

In [15]:
# !pip install psycopg2
#!pip install snowflake-connector-python

In [16]:
symbol = 'btcusdt'
socket = f'wss://stream.binance.com:9443/ws/{symbol}@trade'

In [17]:
# Function to convert Unix timestamp to datetime
def convert_to_datetime(unix_timestamp):
    # Assuming the timestamp is in milliseconds, convert to seconds
    return datetime.fromtimestamp(unix_timestamp / 1000)

In [18]:
#set up database connecton to store the stream data in a table
def get_db_connect():
    connection = psycopg2.connect(
        host = 'localhost',
        database = 'cryptostream',
        user = 'postgres',
        password = 'postgres'
    )
    return connection

conn = get_db_connect()
cursor = conn.cursor()

In [19]:
# Snowflake connection setup
def get_snowflake_connect():
    sf_connection = snowflake.connector.connect(
        user="tolekes",
        password="#Tolexy5038",
        account="nkdtumx-ki12835",
        warehouse="CRYPTOSTREAM",
        database="CRYPTODATABASE",
        schema="CRYPTOSCHEMA"
    )
    return sf_connection

sf_conn = get_snowflake_connect()
sf_cursor = sf_conn.cursor()

In [20]:
from kafka import KafkaConsumer


# Kafka Consumer setup
consumer = KafkaConsumer(
    'binance_trades',
    bootstrap_servers=['localhost:9092'],
    value_deserializer=lambda x: json.loads(x.decode('utf-8'))
)

print("Consumer started. Listening for messages...")


# Function to insert data into PostgreSQL
def insert_into_postgres(event_type, event_date, event_time, symbol, price, quantity):
    insert_query = """
        INSERT INTO binance_data (event_type,event_date, event_time, symbol, price, quantity)
        VALUES (%s, %s, %s, %s, %s, %s)
    """
    cursor.execute(insert_query, (event_type,event_date, event_time, symbol, price, quantity))
    conn.commit()  # Commit the transaction


# Function to Insert data into Snowflake
def insert_into_snowflake(event_type, event_date, event_time, symbol, price, quantity):
    query = """
        INSERT INTO BINANCE_DATA (event_type, event_date, event_time, symbol, price, quantity)
        VALUES (%s, %s, %s, %s, %s, %s)
    """

    sf_cursor.execute(query, (event_type,event_date, event_time, symbol, price, quantity))
    sf_conn.commit()


for message in consumer:
    # Convert received message to DataFrame row
    trade_data = message.value
    
    event_type = trade_data.get('e')
    event_time = trade_data.get('E')
    symbol = trade_data.get('s')
    price = trade_data.get('p')
    quantity = trade_data.get('q')
    
    # Convert event_time to PostgreSQL compatible datetime
    event_datetime = convert_to_datetime(event_time)
    event_date = event_datetime.date()
    event_time = event_datetime.time()

    # Insert the data into the PostgreSQL table
    insert_into_postgres(event_type, event_date, event_time, symbol, price, quantity)

    # Insert the data into the PostgreSQL table
    insert_into_snowflake(event_type, event_date, event_time, symbol, price, quantity)
    
    
# Close the cursor and connection when done
cursor.close()
conn.close()
sf_cursor.close()
sf_conn.close()

Consumer started. Listening for messages...


KeyboardInterrupt: 